#### **Imports**

In [1]:
import warnings;
warnings.filterwarnings('ignore');

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import os, sys
import numpy as np
import random

from  matplotlib.ticker import FuncFormatter # needed to set tickers properly in interactive plots
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")

from ipywidgets import interact
#from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import requests

from settings import PINC_GITHUB_DIR, JSON_CONF_DIR, UPLOAD_DIR
from util.misc import generate_period_list
from util.parse_df import Parse_df
from util.conn_pinc_data import Conn_pinc_data


In [4]:
from stadincijfers import stadincijfers
from gebiedsniveaus.gebiedsniveaus import provincies_dict

#### **Parameters**

In [5]:
all_pinc_periods = True

#### **PinC upload file**

In [6]:
filename = '20210623_Upload crimi en verkeer.xlsx'
file = UPLOAD_DIR + '/' + filename
print(file)

./upload_pinc/20210623_Upload crimi en verkeer.xlsx


#### **Import PinC upload file for validation**

In [7]:
# Read excel file, since there was an issue with (a very limited number)
# of values that were not read properly we do a string transformation (using a converter)
# later on (the opposite) int transformation will be applied
column_list = []
df_column = pd.read_excel(file, engine='openpyxl').columns
for j in df_column:
        column_list.append(j)
converter = {col: str for col in column_list}
#print(column_list)

# Read excel, index_col=0 prevents from adding extra index column
# Sheet_name defaults to 0. Use 1 to access the second sheet, and so on.
# You can also use actual sheet-names (str).
if 'Unnamed' in column_list[0]:
    data_table = pd.read_excel(file, converters=converter, engine='openpyxl', sheet_name=1, index_col=0)
else:
    data_table = pd.read_excel(file, converters=converter, engine='openpyxl', sheet_name=1)

In [8]:
data_table.head()

,period,geolevel,geoitem,v2802_vh_ver_auto,v2802_vh_ver_fiets,v2802_vh_ver_bus,v2802_vh_ver_ongeval_lets
0,2019,gemeente,11001,75,24,4,98
1,2019,gemeente,11002,1866,1082,63,2562
2,2019,gemeente,11004,22,12,0,26
3,2019,gemeente,11005,44,23,1,60
4,2019,gemeente,11007,34,21,0,46


In [9]:
#data_table = data_table[data_table['geolevel']!='gewest']

In [10]:
data_table.head()

,period,geolevel,geoitem,v2802_vh_ver_auto,v2802_vh_ver_fiets,v2802_vh_ver_bus,v2802_vh_ver_ongeval_lets
0,2019,gemeente,11001,75,24,4,98
1,2019,gemeente,11002,1866,1082,63,2562
2,2019,gemeente,11004,22,12,0,26
3,2019,gemeente,11005,44,23,1,60
4,2019,gemeente,11007,34,21,0,46


### **Parse data_table**

In [11]:
parsed_df = Parse_df(data_table)

In [12]:
if all_pinc_periods:
    period_list_pinc_query = parsed_df.determine_years('pinc', _ind_id = 0)
elif not all_pinc_periods:
    period_list_pinc_query = parsed_df.determine_years('table')
else:
    raise Exception("all_pinc_periods requires boolean value")
print(f'Period list for Pinc Query: {period_list_pinc_query}')

List of upload table years: ['2019', '2020']
Period list for Pinc Query: 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019


In [13]:
var_list_pinc_query = parsed_df.determine_indicators()
print(f'Indicator list for PinC Query: {var_list_pinc_query}')

Indicator list for PinC Query: v2802_vh_ver_auto,v2802_vh_ver_fiets,v2802_vh_ver_bus,v2802_vh_ver_ongeval_lets


In [14]:
geolevel = parsed_df.determine_geolevel()
print(f'Geolevel for PinC Query: {geolevel}')

Geolevel for PinC Query: gemeente


### **PinC Query**

In [15]:
# Instantiate object from class
pinc = stadincijfers("provincies")

In [16]:
pinc_table = pinc.selectiontableasDataframe(var_list_pinc_query, geolevel=geolevel, periodlevel='year',period=period_list_pinc_query)

In [17]:
pinc_table.head()

,Geo,Perioden,auto-ongevallen,fietsongevallen,busongevallen,verkeersongevallen met letsel - dummy
0,Aalst,2009,364.0,105,10,437
1,Aalst,2010,350.0,113,11,431
2,Aalst,2011,367.0,120,4,445
3,Aalst,2012,291.0,109,6,368
4,Aalst,2013,295.0,102,2,366


In [18]:
if pinc_table.columns[0] != 'Geo':
    pinc_table.rename(columns={pinc_table.columns[0]: 'Geo'}, inplace = True)

## ***Interactive check framework:***

In [19]:
connection = Conn_pinc_data(pinc_table,data_table)

In [20]:
var_dic = connection.cols_to_dict()
geo_dic = Conn_pinc_data.reversed_level_code_dict(parsed_df.geolevel)

In [21]:
var = random.choice(list(var_dic.keys()))
print(f'Random variable: {var}')
geo = random.choice(list(geo_dic.keys()))
print(f'Random location: {geo}')

Random variable: busongevallen
Random location: Sint-Genesius-Rode


In [22]:
pinc_table.head()

,Geo,Perioden,auto-ongevallen,fietsongevallen,busongevallen,verkeersongevallen met letsel - dummy
0,Aalst,2009,364.0,105,10,437
1,Aalst,2010,350.0,113,11,431
2,Aalst,2011,367.0,120,4,445
3,Aalst,2012,291.0,109,6,368
4,Aalst,2013,295.0,102,2,366


#### **Interactive plots:**

##### **Set additive factor/constant for visually comparing upload and pinc number sequences. Set to 0 and re-run from here before running the outlier analysis (see issue 1)  **

In [23]:
constant = 0

In [24]:
# int transform changed to float transfrom in lambdas
# to do: make constant (to make lines visually different) dynamic
def draw_figure(var= var ,geo= geo):
    tmp_df = pd.DataFrame()
    
    upload_var = var_dic.get(var)
    upload_geo = geo_dic.get(geo)
    
    tmp_df_pinc = pinc_table[pinc_table['Geo']==geo][['Perioden','Geo',var]]
    tmp_df_pinc['Geo'] = tmp_df_pinc['Geo'].apply(lambda x: str(x) + '_PinC')
    tmp_df_pinc[var] = tmp_df_pinc[var].apply(lambda x: None if ((not x) or (x=='-') or (x=='x') or (x=='?')) else (float(x) + constant) )  # We add one to ease comparison in plot
    tmp_df = tmp_df.append(tmp_df_pinc)
    #print('PinC Dataframe:')
    #print(tmp_df.head())
    
    tmp_df_upload = data_table[data_table['geoitem']==upload_geo][['period','geoitem',upload_var]]
    tmp_df_upload = tmp_df_upload.rename(columns={'geoitem':'Geo', 'period': 'Perioden', upload_var : var})
    tmp_df_upload['Geo'] = tmp_df_upload['Geo'].apply(lambda x: str(x) + '_Upload')
    tmp_df_upload[var] = tmp_df_upload[var].apply(lambda x: float(x))
    tmp_df = tmp_df.append(tmp_df_upload)
    #print('Upload Dataframe:')
    #print(tmp_df.head())
    
    tmp_df['Perioden'] = tmp_df['Perioden'].apply(lambda x: int(x))
    #print(tmp_df['Perioden'].unique())
    sns.lineplot(data=tmp_df, x='Perioden',y= var,hue='Geo', style='Geo', markers=True, dashes=False);
    # Format year on xticks to int (instead of float):
    plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: int(x)));
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.);


In [25]:
#draw_figure()

In [26]:
list_vars = sorted(list(var_dic.keys()))
list_geos = sorted(list(geo_dic.keys()))  # to do: do an alphabetic sort first

In [27]:
# to do: make better label names
interact(draw_figure, var=list_vars,geo=list_geos);

interactive(children=(Dropdown(description='var', index=1, options=('auto-ongevallen', 'busongevallen', 'fiets…

#### **Iterative outlier analysis**

##### **Overall outlier analysis (make a choice between (A.) automatic generation of comparison years or (B.) setting comparison years yourself):**

In [28]:
# A. Random generation of comparison years (starting from oldest year in upload file):
upload_year = parsed_df.years[0]
print(f'Upload year: {upload_year}')
if str((int(upload_year)-1)) in parsed_df.pinc_q_years :
    pinc_year = int(upload_year)-1
else:
    pinc_year = random.choice(parsed_df.pinc_q_years)
print(f'Pinc year: {pinc_year}')

Upload year: 2019
Pinc year: 2018


In [29]:
# B. Set comparison years:
upload_year = 2020
pinc_year = 2019

In [49]:
tmp_df = pd.DataFrame()
tel = 0
for var in list(var_dic.keys()):


    upload_var = var_dic.get(var)

    tmp_df_pinc = pinc_table[pinc_table['Perioden']==str(pinc_year)][['Geo',var]]
    tmp_df_pinc[var] = tmp_df_pinc[var].apply(lambda x: None if ((not x) or (x=='-') or (x=='x') or (x=='?')) else (float(x) + constant) )  # We add one to ease comparison in plot
    tmp_df_pinc = tmp_df_pinc.rename(columns={'Geo':'Geo_pinc',var: '_pinc_' + str(pinc_year)})
    tmp_df_pinc['geo_code_pinc'] = tmp_df_pinc['Geo_pinc'].map(geo_dic)


    tmp_df_upload = data_table[data_table['period']==str(upload_year)][['geoitem',upload_var]]
    tmp_df_upload = tmp_df_upload.rename(columns={'geoitem':'geo_code_upload', upload_var : '_upload_' + str(upload_year)})
    tmp_df_upload['_upload_' + str(upload_year)] = tmp_df_upload['_upload_' + str(upload_year)].apply(lambda x: float(x))

    merged_df = tmp_df_pinc.merge(tmp_df_upload, left_on='geo_code_pinc', right_on='geo_code_upload')
    merged_df['diff'] = abs(merged_df['_pinc_' + str(pinc_year)]-merged_df['_upload_' + str(upload_year)])/merged_df['_pinc_' + str(pinc_year)]
    merged_df['Indicator'] = var + '_P' + str(pinc_year) + '_U' + str(upload_year)
    
    ## Uncomment next line for some validation
    #print(f'Variable: {var}, Shape: {merged_df.shape}')
    tel += merged_df.shape[0]
    tmp_df = tmp_df.append(merged_df)

## Uncomment two next lines for some counters to be used for validation:    
#tel_t = len(list(var_dic.keys())*merged_df.shape[0])     
#print(f'Cumulated no. rows: {tel} vs theoretical no. rows: {tel_t}')

# Uncomment second line if you want to ignore diff=inf as a result from division by zero
tmp_df.sort_values(by=['diff'], ascending=False)[['Geo_pinc', '_pinc_' + str(pinc_year), '_upload_' + str(upload_year),'diff','Indicator']].head(30)
#tmp_df[~(tmp_df['diff']==np.inf)].sort_values(by=['diff'], ascending=False)[['Geo_pinc', '_pinc_' + str(pinc_year), '_upload_' + str(upload_year),'diff','Indicator']].head(30)

,Geo_pinc,_pinc_2019,_upload_2020,diff,Indicator
31,Blankenberge,0.0,1.0,inf,busongevallen_P2019_U2020
72,Etterbeek,0.0,3.0,inf,busongevallen_P2019_U2020
242,Schilde,0.0,1.0,inf,busongevallen_P2019_U2020
244,Sint-Agatha-Berchem,0.0,2.0,inf,busongevallen_P2019_U2020
183,Maasmechelen,0.0,1.0,inf,busongevallen_P2019_U2020
181,Maarkedal,0.0,1.0,inf,busongevallen_P2019_U2020
149,Kraainem,0.0,1.0,inf,busongevallen_P2019_U2020
99,Heers,0.0,5.0,inf,fietsongevallen_P2019_U2020
33,Boechout,0.0,2.0,inf,busongevallen_P2019_U2020
82,Geraardsbergen,0.0,1.0,inf,busongevallen_P2019_U2020


##### **Univariate outlier analysis:**

In [58]:
def show_outliers(var = var, upload_year = upload_year, pinc_year = pinc_year):
    upload_var = var_dic.get(var)
    
    tmp_df_pinc = pinc_table[pinc_table['Perioden']==str(pinc_year)][['Geo',var]]
    tmp_df_pinc[var] = tmp_df_pinc[var].apply(lambda x: None if ((not x) or (x=='-') or (x=='x') or (x=='?')) else (float(x) + constant) )  # We add one to ease comparison in plot
    tmp_df_pinc = tmp_df_pinc.rename(columns={'Geo':'Geo_pinc',var: var + '_pinc_' + str(pinc_year)})
    tmp_df_pinc['geo_code_pinc'] = tmp_df_pinc['Geo_pinc'].map(geo_dic)


    tmp_df_upload = data_table[data_table['period']==str(upload_year)][['geoitem',upload_var]]
    tmp_df_upload = tmp_df_upload.rename(columns={'geoitem':'geo_code_upload', upload_var : var + '_upload_' + str(upload_year)})
    tmp_df_upload[var + '_upload_' + str(upload_year)] = tmp_df_upload[var + '_upload_' + str(upload_year)].apply(lambda x: float(x))

    merged_df = tmp_df_pinc.merge(tmp_df_upload, left_on='geo_code_pinc', right_on='geo_code_upload')
    merged_df['diff'] = abs(merged_df[var + '_pinc_' + str(pinc_year)]-merged_df[var + '_upload_' + str(upload_year)])/merged_df[var + '_pinc_' + str(pinc_year)]
    #return merged_df.sort_values(by=['diff'], ascending=False)[['Geo_pinc', var + '_pinc_' + str(pinc_year), var + '_upload_' + str(upload_year),'diff']].head(20)
    return merged_df[~(merged_df['diff']==np.inf)].sort_values(by=['diff'], ascending=False)[['Geo_pinc', var + '_pinc_' + str(pinc_year), var + '_upload_' + str(upload_year),'diff']].head(20)

In [59]:
#show_outliers()

In [60]:
list_upload_years = parsed_df.years
list_pinc_years = parsed_df.pinc_q_years

In [61]:
# to do: make better label names
interact(show_outliers, var=list_vars,upload_year = list_upload_years, pinc_year = list_pinc_years);

interactive(children=(Dropdown(description='var', index=3, options=('auto-ongevallen', 'busongevallen', 'fiets…